
# Projeto: Regressão Linear Simples — Base de Dados de Futebol

**Arquivo da base:** `dados-completos-Ituano.csv`

## Integrantes do Grupo
> Preencha abaixo com os dados dos integrantes (nome completo, RA, e-mail):
- Nome 1 — RA — e-mail
- Nome 2 — RA — e-mail
- Nome 3 — RA — e-mail
- Nome 4 — RA — e-mail

---


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# 1) Importar base
df = pd.read_csv("dados-completos-Ituano.csv")
df.head()


## Inspeção Inicial da Base

In [ ]:

df.info()


In [ ]:

df.describe(include="all")



## Seleção das Variáveis (X, Y)

Para aplicar a **regressão linear simples**, selecionamos automaticamente o par de colunas **numéricas** com **maior correlação linear (em valor absoluto)**, evitando correlação de uma coluna com ela mesma.

> Você pode alterar manualmente as colunas definindo `X_col` e `Y_col` conforme desejar.


In [ ]:

# Selecionar colunas numéricas
numeric_df = df.select_dtypes(include=[np.number]).copy()

# Verificar se há ao menos 2 colunas numéricas
assert numeric_df.shape[1] >= 2, "É necessário pelo menos 2 colunas numéricas para regressão linear simples."

# Matriz de correlação (apenas numérica)
corr = numeric_df.corr(numeric_only=True)

# Encontrar o par com maior |correlação|
corr_unstack = corr.where(~np.eye(corr.shape[0], dtype=bool))
abs_corr_long = corr_unstack.abs().stack().sort_values(ascending=False)
(Y_col, X_col) = abs_corr_long.index[0]  # Y primeiro, X segundo

Y_col, X_col, abs_corr_long.iloc[0]


## Ajuste do Modelo de Regressão Linear Simples

In [ ]:

# Preparar dados
X = numeric_df[[X_col]].copy()
X = sm.add_constant(X)  # intercepto
Y = numeric_df[Y_col]

# Remover valores ausentes
valid = X.notna().all(axis=1) & Y.notna()
X = X[valid]
Y = Y[valid]

# Ajustar modelo OLS
model = sm.OLS(Y, X).fit()
model.summary()


## Intervalos de Confiança (IC) dos Parâmetros

In [ ]:

conf_int = model.conf_int(alpha=0.05)
conf_int.columns = ["IC_2.5%", "IC_97.5%"]
conf_int



**Descrição das saídas principais:**
- **coef**: estimativas dos parâmetros (intercepto e inclinação).
- **std err**: erro padrão das estimativas.
- **t**, **P>|t|**: estatística de teste t e p-valor (teste de hipótese H0: parâmetro = 0).
- **[0.025, 0.975]**: intervalos de confiança de 95% para os parâmetros.
- **R-squared**: coeficiente de determinação; proporção da variabilidade de `Y` explicada por `X`.
- **Adj. R-squared**: R² ajustado (para múltiplas variáveis, aqui é igual/semelhante).
- **F-statistic** e **Prob (F-statistic)**: teste conjunto do modelo (para regressão simples
  coincide com o teste t do coeficiente de inclinação).
- **Omnibus**, **Jarque-Bera**: testes de normalidade dos resíduos.
- **Durbin-Watson**: autocorrelação dos resíduos (ideal próximo de 2).


## Gráfico de Dispersão com a Reta de Regressão

In [ ]:

plt.figure()
plt.scatter(X[X_col], Y)
x_vals = np.linspace(X[X_col].min(), X[X_col].max(), 200)
y_hat = model.params["const"] + model.params[X_col] * x_vals
plt.plot(x_vals, y_hat)
plt.xlabel(X_col)
plt.ylabel(Y_col)
plt.title(f"Dispersão com reta de regressão: {Y_col} ~ {X_col}")
plt.show()


## Diagnóstico Rápido dos Resíduos (Opcional)

In [ ]:

res = model.resid
fitted = model.fittedvalues

# Plot dos resíduos vs valores ajustados
plt.figure()
plt.scatter(fitted, res)
plt.axhline(0)
plt.xlabel("Valores ajustados")
plt.ylabel("Resíduos")
plt.title("Resíduos vs. Ajustados")
plt.show()

# Histograma dos resíduos
plt.figure()
plt.hist(res.dropna(), bins=20)
plt.title("Histograma dos Resíduos")
plt.xlabel("Resíduo")
plt.ylabel("Frequência")
plt.show()
